In [70]:
import pandas as pd
from datasets import Dataset
import re
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid
from tqdm import tqdm



In [71]:
qdrant_set=Dataset.from_file('YOUR-PATH-HERE')

In [72]:
selected_columns=qdrant_set.select_columns(["doctor_title", "doctor_speciality", "question_content", "question_answer"])
df_qdrant=selected_columns.to_pandas()
df_qdrant_2 = selected_columns.to_pandas()

In [73]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [96]:
df_qdrant['text'] = df_qdrant.apply(
    lambda row: f"{clean_text(row['question_content'])}", 
    axis=1
)

In [100]:
df_qdrant['text'].head(3)

0    merhaba doktor bey 20 gun once kil donmesinden...
1    merhaba doktor hanım benim bugün adettimin 4 g...
2    sayın doktor abla ben 14 yaşında bi öğrenciyim...
Name: text, dtype: object

In [97]:
sentences = df_qdrant["text"].tolist()
metadata = df_qdrant_2[["doctor_title", "doctor_speciality", "question_content", "question_answer"]].to_dict(orient="records")

In [107]:
sentences

['merhaba doktor bey 20 gun once kil donmesinden ameliyat oldum ve iltihap oldugu soylendi 1 hafta once tekrar bir operasyon gecirdim suan ameliyat bolgesi dikissiz ve hergün sicak su ile yikanmasi gerektgini soylediler sizce ne yapmaliyim ve yaranin kapanmasi zaman alir mi yardimci olursaniz tesekkur ederim',
 'merhaba doktor hanım benim bugün adettimin 4 günü ve ben mastübasyon yaptım ve üzerimde iç çamaşırı ve ped vardı bu şekilde kızlık zarı bozulur mu ve vajinanın içine hiç bir şey sokmadım ve mastübasyon yaptıktan 10 dakika sonra lavaboya gittiğimde kan vardı ve pıhtı şekilde kanamam olmuştu ve kan da vardı ama ben ve karnım şiddetli bir şekilde ağrıyor adet kanımın bu şekilde pıhtılı gelmesi normal mi ve bu şekilde kızlık zarı bozulur mu ve sadece baskı vardı ama vajinanın içine hiç bir şey sokmadım iç çamaşırı ve ped vardı teşekkürler',
 'sayın doktor abla ben 14 yaşında bi öğrenciyim benim üst taraftaki köpek dişleri çıkmadı normalde 11 12 yaşında çıkması gerekiyormuş neden çı

In [99]:
model = SentenceTransformer('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
embeddings = model.encode(sentences, show_progress_bar=True)

Batches: 100%|██████████| 1173/1173 [17:24<00:00,  1.12it/s]


In [101]:
client = QdrantClient("http://localhost", port=6333)

In [102]:
collection_name = "doctor_questions_2"
vector_size = len(embeddings[0])
print(vector_size)
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
)

768


/tmp/ipykernel_31569/2755409553.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [103]:
# Koleksiyonun beklediği boyut
collection_info = client.get_collection("doctor_questions_2")
expected_size = collection_info.config.params.vectors.size

print(expected_size)
# Gönderilen embedding boyutu
actual_size = len(embeddings[0])
assert actual_size == expected_size, f"Boyut uyumsuz: {actual_size} != {expected_size}"
print(actual_size)

768
768


In [104]:
from qdrant_client import models
from tqdm import tqdm

BATCH_SIZE = 200  # 200-500 arası deneyin
points = []

for idx, (vector, meta) in enumerate(tqdm(zip(embeddings, metadata), total=len(embeddings)), start=1):
    points.append(models.PointStruct(
        id=idx,  # Sıralı ID veya uuid.uuid4().hex
        vector=vector.tolist() if hasattr(vector, 'tolist') else vector,
        payload=meta  # String boyutunu sınırla
    ))
    
    # Batch dolunca gönder
    if len(points) >= BATCH_SIZE:
        client.upsert(
            collection_name=collection_name,
            points=points,
            wait=False  # Performans için
        )
        points = []

# Kalan noktaları gönder
if points:
    client.upsert(
        collection_name=collection_name,
        points=points,
        wait=True  # Son batch'i beklet
    )

100%|██████████| 37527/37527 [00:55<00:00, 678.76it/s]
